In [11]:
import pyrealsense2 as rs
import numpy as np
import cv2
import os.path
import mediapipe as mp
import time
import os
from support.funcs import *

try:
    os.remove('posout.npy')
except:
    pass
h=720 
w=1280
fps=30
windowscale=0.6

LS, LE, LW, RS, RE, RW, TR = [], [], [], [], [], [], []
RI,LI=[],[]

mp_holistic = mp.solutions.holistic
holistic_model = mp_holistic.Holistic(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)
 
# Initializing the drawing utils for drawing the facial landmarks on image
mp_drawing = mp.solutions.drawing_utils

pc = rs.pointcloud()


try:
    # Create pipeline
    pipeline = rs.pipeline()

    # Create a config object
    config = rs.config()

    # Tell config that we will use a recorded device from file to be used by the pipeline through playback.
    rs.config.enable_device_from_file(config, r'C:\Users\arpan\OneDrive\Documents\internship\bags\arot.bag', repeat_playback=False)

    # Configure the pipeline to stream the depth stream
    # Change this parameters according to the recorded bag file resolution
    config.enable_stream(rs.stream.depth, rs.format.z16, fps)
    config.enable_stream(rs.stream.color, rs.format.rgb8, fps)

    # Start streaming from file
    profile=pipeline.start(config)
    
    # Create colorizer object
    colorizer = rs.colorizer()

    #Needed so frames don't get dropped during processing:
    profile.get_device().as_playback().set_real_time(False)

    align_to = rs.stream.color
    align = rs.align(align_to)

    timestamps=[]
        
    c=0
    # Streaming loop
    while True:

        frame_present, frameset = pipeline.try_wait_for_frames()
    
        #End loop once video finishes
        if not frame_present:
            break

        # Wait for a coherent pair of frames: depth and color
        frames = pipeline.wait_for_frames()
        aligned_frames = align.process(frames)
        depth_frame = frames.get_depth_frame()
        color_frame = frames.get_color_frame()
        if not depth_frame or not color_frame:
            continue
        
        # get timestamp of frame
        timestamps.append((rs.frame.get_frame_metadata(depth_frame,rs.frame_metadata_value.time_of_arrival))/1000)

        # Get aligned frames
        aligned_depth_frame = aligned_frames.get_depth_frame() 
        color_frame = aligned_frames.get_color_frame()

        # Convert images to numpy arrays
        depth_image = np.asanyarray(aligned_depth_frame.get_data()) 
        color_image = np.asanyarray(color_frame.get_data()) 

        # Making predictions using holistic model
        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        color_image.flags.writeable = False
        results = holistic_model.process(color_image)
        color_image.flags.writeable = True

        #converting color image to BGR
        color_image = cv2.cvtColor(color_image, cv2.COLOR_RGB2BGR)

        # Drawing the pose landmarks
        mp_drawing.draw_landmarks(
        color_image,
        results.pose_landmarks,
        mp_holistic.POSE_CONNECTIONS)

        # Apply colormap on depth image (image must be converted to 8-bit per pixel first)
        depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET)

        depth_colormap_dim = depth_colormap.shape
        color_colormap_dim = color_image.shape
        
        try:
            dic = {}
            for mark, data_point in zip(mp_holistic.PoseLandmark, results.pose_landmarks.landmark):
                dic[mark.value] = dict(landmark = mark.name, 
                    x = data_point.x,
                    y = data_point.y)        
            try:
                LS.append([dic[11]['x']*w,dic[11]['y']*h])
            except:
                LS.append(np.nan)
            try:
                LE.append([dic[13]['x']*w,dic[13]['y']*h])
            except:
                LE.append(np.nan)
            try:
                LW.append([dic[15]['x']*w,dic[15]['y']*h])
            except:
                LW.append(np.nan)
            try:
                RS.append([dic[12]['x']*w,dic[12]['y']*h])
            except:
                RS.append(np.nan)
            try:
                RE.append([dic[14]['x']*w,dic[14]['y']*h])
            except:
                RE.append(np.nan)
            try:
                RW.append([dic[16]['x']*w,dic[16]['y']*h])
            except:
                RW.append(np.nan)
            
            try:
                Smid=midpoint([dic[11]['x']*w,dic[11]['y']*h],[dic[12]['x']*w,dic[12]['y']*h])
                perpx=int(Smid[0])
                perpy=(int(Smid[1])+25)

                cv2.circle(color_image,(perpx,perpy) , 5, (0, 0, 255), 2)
                TR.append([perpx,perpy])     #in uv format  
            except:
                TR.append(np.nan)

            try:
                RI.append([dic[20]['x']*w,dic[20]['y']*h])
                LI.append([dic[19]['x']*w,dic[19]['y']*h])

                cv2.putText(color_image, str(int(c)), (50, 70), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
                draw_box(color_image,LS[c],LE[c])
                draw_box(color_image,RS[c],RE[c])
                draw_box(color_image,LE[c],LW[c])
                draw_box(color_image,RE[c],RW[c])
                draw_box(color_image,RW[c],([dic[20]['x']*w,dic[20]['y']*h]),(255,0,255),40)
                draw_box(color_image,LW[c],([dic[19]['x']*w,dic[19]['y']*h]),(0,255,255),40) 
            except:
                RI.append(np.nan)
                LI.append(np.nan)
        except:
            LS.append(np.nan)
            LE.append(np.nan)
            LW.append(np.nan)

            RS.append(np.nan)
            RE.append(np.nan)
            RW.append(np.nan)

            TR.append(np.nan) 
            RI.append(np.nan)
            LI.append(np.nan)
            pass
        
        color_image=cv2.resize(color_image, (int(w*windowscale),int(h*windowscale)))
        depth_colormap=cv2.resize(depth_colormap, (int(w*windowscale),int(h*windowscale)))

        # If depth and color resolutions are different, resize color image to match depth image for display
        if depth_colormap_dim != color_colormap_dim:
            resized_color_image = cv2.resize(color_image, dsize=(depth_colormap_dim[1], depth_colormap_dim[0]), interpolation=cv2.INTER_AREA)
            images = np.hstack((resized_color_image, depth_colormap))
        else:
            images =np.hstack((color_image, depth_colormap))
        
        # Show images
        cv2.putText(color_image, str(int(c)), (50, 70), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
        cv2.namedWindow('RealSense', cv2.WINDOW_AUTOSIZE)
        cv2.imshow('RealSense', images)
        if cv2.waitKey(5) & 0xFF == ord('q'):
            break

        mapped_frame = color_frame
        pc.map_to(mapped_frame)
        
        try:
            points = pc.calculate(aligned_depth_frame)
            v = points.get_vertices()
            verts = np.asanyarray(v).view(np.float32)
            xyzpos=verts.reshape(h,w, 3)  # xyz
            xyzpos=xyzpos.astype(np.float16)
            with open('posout.npy', 'ab') as f:
                np.save(f,xyzpos)   
                # print(c)         
        except:
            print(type(v))
        
        c+=1

finally:

    # Stop streaming
    pipeline.stop()
    cv2.destroyAllWindows()
    print(c)

42


In [12]:
import pandas as pd
import numpy as np

land_marks={'LS':LS,'LE':LE,'LW':LW,'RS':RS,'RE':RE,'RW':RW, 'TR':TR}

pos=[]
with open('posout.npy', 'rb') as f:
    for i in range(c):
        y=np.load(f)
        pos.append(y)

pos=np.array(pos)
print(pos.shape)
df=pd.DataFrame()
xyz=['_x','_y','_z']

df['epoch_time']=pd.Series(timestamps)

for key,value in land_marks.items():    
    for j in range(3):
        data=[]
        for i in range(len(pos)):
            try:
                x=pos[i][int((value[i][1]))][int((value[i][0]))][j]
                data.append(x)
            except:
                continue
        df[key+xyz[j]]=pd.Series(data)
    
df

(42, 720, 1280, 3)


,epoch_time,LS_x,LS_y,LS_z,LE_x,LE_y,LE_z,LW_x,LW_y,LW_z,...,RS_z,RE_x,RE_y,RE_z,RW_x,RW_y,RW_z,TR_x,TR_y,TR_z
0,1.686732e+09,0.217529,-0.538086,2.480469,0.188843,-0.307861,2.498047,0.322021,-0.119385,2.529297,...,2.439453,0.296387,-0.146851,2.523438,0.528320,-0.007950,2.542969,0.208984,-0.411621,2.445312
1,1.686732e+09,0.205078,-0.536621,2.462891,0.192993,-0.307373,2.517578,0.323730,-0.122803,2.542969,...,2.445312,0.298340,-0.146606,2.517578,0.532227,-0.005135,2.535156,0.197266,-0.412354,2.433594
2,1.686732e+09,0.194214,-0.539551,2.462891,0.194824,-0.305908,2.505859,0.327637,-0.124939,2.529297,...,2.462891,0.294922,-0.148926,2.511719,0.531250,-0.002335,2.529297,0.192017,-0.423828,2.468750
3,1.686732e+09,0.188354,-0.557129,2.457031,0.193359,-0.311768,2.486328,0.330566,-0.121948,2.468750,...,2.462891,0.274170,-0.146729,2.474609,0.500488,-0.002335,2.529297,0.180786,-0.427490,2.427734
4,1.686732e+09,0.184326,-0.585449,2.474609,0.202148,-0.318115,2.492188,0.437500,-0.183472,3.214844,...,2.445312,0.218994,-0.163208,2.562500,0.440918,-0.013451,2.523438,0.174438,-0.455811,2.451172
5,1.686732e+09,0.178955,-0.577637,2.439453,0.177856,-0.334473,2.621094,0.348145,-0.157959,2.937500,...,2.433594,0.113708,-0.223633,2.480469,0.327881,-0.041046,2.511719,0.165894,-0.481689,2.445312
6,1.686732e+09,0.171753,-0.596191,2.451172,0.131226,-0.354248,2.554688,0.274658,-0.175903,2.912109,...,2.439453,0.028946,-0.247681,2.474609,0.229126,-0.068909,2.517578,0.147827,-0.508301,2.457031
7,1.686732e+09,0.175415,-0.624023,2.542969,0.125244,-0.378662,2.607422,0.272949,-0.183350,2.929688,...,2.439453,-0.044434,-0.289795,2.462891,0.146118,-0.092224,2.474609,0.123718,-0.525879,2.462891
8,1.686732e+09,0.169067,-0.645996,2.574219,0.122681,-0.399658,2.613281,0.265625,-0.204468,2.953125,...,2.439453,-0.101624,-0.333984,2.468750,0.042480,-0.100159,2.468750,0.100037,-0.554199,2.480469
9,1.686732e+09,0.160156,-0.664551,2.613281,0.128540,-0.424072,2.675781,0.253662,-0.205933,2.929688,...,2.451172,-0.147217,-0.374756,2.505859,-0.038910,-0.124023,2.457031,0.079956,-0.585938,2.535156


In [13]:
startflag=0
before_occ=0
do=False
box_dic={'lub': [LS,LE,['LS','LE']],'rub': [RS,RE,['RS','RE']],
         'llb': [LE,LW,['LE','LW']],'rlb': [RE,RW,['RE','RW']],
         'lhb': [LW,LI,['LW']],'rhb': [RW,RI,['RW']]}

for k,j in land_marks.items():
    for key,values in box_dic.items():
        for i in range(c):
            r= 40 if key == 'lhb' or key == 'rhb' else 30
            try:
                if point_in_quad(j[i],draw_box(color_image,values[0][i],values[1][i],(0,0,1),r)) and k not in values[2]:
                    if startflag == 0:
                        startflag = i
                        before_occ=startflag-1 # before occlusion
                    for p in values[2]:
                        if df[k+'_z'].tolist()[before_occ]>df[p+'_z'].tolist()[before_occ]:
                            print(k,'is occluded by', key, p , 'at frame', i) 
                            # df.loc[i,k+'_x']=df.loc[before_occ,k+'_y']
                            # df.loc[i,k+'_y']=df.loc[before_occ,k+'_x']
                            df.loc[i,k+'_z']=df.loc[before_occ,k+'_z']
                    do=True
            except:
                pass
            else:
                do=False
        if do:
            startflag=0

LS is occluded by rub RS at frame 5
LS is occluded by rub RS at frame 6
LE is occluded by rub RS at frame 0
LE is occluded by rub RS at frame 1
LE is occluded by rub RS at frame 2
LE is occluded by rub RS at frame 3
LE is occluded by rub RS at frame 4
LE is occluded by rub RS at frame 5
LE is occluded by rub RS at frame 6
LW is occluded by rub RS at frame 0
LW is occluded by rub RE at frame 0
LW is occluded by rub RS at frame 1
LW is occluded by rub RE at frame 1
LW is occluded by rub RS at frame 2
LW is occluded by rub RE at frame 2
LW is occluded by rub RS at frame 3
LW is occluded by rub RE at frame 3
LW is occluded by rlb RE at frame 0
LW is occluded by rlb RW at frame 0
LW is occluded by rlb RE at frame 1
LW is occluded by rlb RW at frame 1
LW is occluded by rlb RE at frame 2
LW is occluded by rlb RW at frame 2
LW is occluded by rlb RE at frame 3
LW is occluded by rlb RW at frame 3
LW is occluded by rlb RE at frame 4
LW is occluded by rlb RW at frame 4
LW is occluded by rlb RE at 

In [14]:
df.rename(columns={'LS_x':'ls_x','LE_x':'le_x','LW_x':'lw_x','RS_x':'rs_x','RE_x':'re_x','RW_x':'rw_x','TR_x':'tr_x',
                   'LS_y':'ls_y','LE_y':'le_y','LW_y':'lw_y','RS_y':'rs_y','RE_y':'re_y','RW_y':'rw_y','TR_y':'tr_y',
                   'LS_z':'ls_z','LE_z':'le_z','LW_z':'lw_z','RS_z':'rs_z','RE_z':'re_z','RW_z':'rw_z','TR_z':'tr_z'},inplace=True)
df.to_csv('mpipe.csv', index=False)
print(df.columns)


Index(['epoch_time', 'ls_x', 'ls_y', 'ls_z', 'le_x', 'le_y', 'le_z', 'lw_x',
       'lw_y', 'lw_z', 'rs_x', 'rs_y', 'rs_z', 're_x', 're_y', 're_z', 'rw_x',
       'rw_y', 'rw_z', 'tr_x', 'tr_y', 'tr_z'],
      dtype='object')
